In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t
import nglview as nv
from simtk import unit
import numpy as np

/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1170: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


_ColormakerRegistry()

# Cálculo de distancias

El cálculo de distancias en MolModMT se puede hacer actualmente recurriendo a dos motores: el propio conjunto de librerías de MolMoMT programadas para tal efecto y las encontradas en la librería MDTraj (en el futuro se añadirán MDAnalysis y Pytraj). La opción por defecto, si no se especifica otra cosa, es usar el código propio de MolModMT. Si por algún motivo se prefiere realizar los cálculos empleando métodos de MDTraj, se puede añadir el argumento de entrada `engine='MDTraj'` a cada uno de los métodos que aparecen a continuación.

## Distancias simples

MolModMT acepta trabajar con matrices de coordenadas espaciales como la forma más simple de codificar las posiciones de un modelo molecular de átomos iguales, sin información topológica. Es lo que MolModMT denomina objeto `XYZ`:

In [ ]:
model_A = np.zeros([6,3], dtype='float64') * unit.angstroms

In [ ]:
m3t.get_form(model_A)

In [ ]:
m3t.get(model_A, n_frames=True, n_atoms=True)

El objeto XYZ debe ser un tensor de tipo array de numpy de rango 3 o 2 según sea: [índice_frame, indice_átomo, coordenada] o [índice_atómo, coordenada], donde coordenada es x, y o z. Es obligatorio que este array de numpy lleve unidades de longitud.

In [ ]:
model_A = np.zeros([2,4,3], dtype='float64') * unit.nanometers

In [ ]:
m3t.get(model_A, n_frames=True, n_atoms=True)

### Distancia simple entre puntos

El caso más sencillo que podemos resolver en cuanto al cálculo de distancias es el de la distancia entre los puntos de un conjunto, o entre dos conjuntos distintos de puntos en el espacio, sin condiciones de contorno periódicas. 

#### Caso 1

Supongamos un conjunto único conjunto de tres puntos con coordenadas que adjudicaremos arbitrariamente a modo de ejemplo:

In [ ]:
set_A = np.zeros([1,3,3], dtype='float64') * unit.nanometers

set_A[0,0,:] = [0, 2, -1] * unit.nanometers
set_A[0,1,:] = [-1, 1, 1] * unit.nanometers
set_A[0,2,:] = [-2, 0, 1] * unit.nanometers

Podemos calcular la distancia entre los elementos de `set_A` como:

In [ ]:
distances = m3t.distance(item_1=set_A)

In [ ]:
distances

El resultado es un tensor de rango 3 donde el primer rango es el índice de frame, en este caso no hay más que uno, y el segundo y el tercer rango corresponden a los indices de los elementos de `set_A`:

In [ ]:
distances.shape

De tal manera que la distancia entre el elemento 0-ésimo y el 2-ésimo en el único frame 0-ésimo es:

In [ ]:
print(distances[0,0,2])

Si quisíeramos haber calculado únicamente la distancia entre esos dos elementos podríamos haber recurrido a:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=0, selection_2=2)

In [ ]:
print(distances)

Observa que la forma del array es un tensor de rango 3 donde en este caso todas las dimensiones son iguales a 1:

In [ ]:
distances.shape

Esto es porque en este caso sólo tenemos un frame (primer rango) con dos listas de elementos (segundo y tercer rango): `selection_1` y `selection_2`. Y en este caso ambas listas contienen un único elemento, el 0-ésimo en el primer caso y el 2-ésimo en el segundo. Veamos ahora qué pasa si queremos calcular la distancia entre el elemento 0-ésimo y los otros dos:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=0, selection_2=[1,2])

In [ ]:
distances.shape

El tensor de distancias tiene ahora dimensión 2 en el último rango. ¿Cómo entonces por ejemplo veríamos cúal es la distancia entre el elemento 0-ésimo y el 2-ésimo en el único frame que tenemos según el actual objeto `distances`? 

In [ ]:
print(distances[0,0,1])

Para mayor claridad podemos también obtener el resultado del método `molmodmt.distance` como diccionario, donde ahora sí podemos recurrir al índice de elemento expresado en los argumentos `selection_1` y `selection_2`:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=0, selection_2=[1,2], output_form='dict')

In [ ]:
print(distances)

En realidad el objeto `distances` es ahora un diccionario de diccionarios para poder rápidamente conocer las distancias entre dos elementos en función del índice de frame. Veamos cómo acceder a la lista de distancias para los diferentes frames (aquí sólo uno) entre el elemento 0-ésimo incluido en `selection_1` y el 2-ésimo incluido en `selection_2`:

In [ ]:
print(distances[0][2])

#### Caso 2

Supongamos ahora que tenemos dos conjuntos de puntos en el espacio en tres instantes de tiempo: `set_A` con tres elementos y `set_B` con dos.

In [ ]:
# Set_A inicializado a cero:
set_A = np.zeros([3,3,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_A para los tres frames 
set_A[0,0,:] = [0, 2, -1] * unit.nanometers
set_A[1,0,:] = [1, 2, -1] * unit.nanometers
set_A[2,0,:] = [0, 2, -1] * unit.nanometers

## coordenadas del segundo elemento de Set_A para los tres frames
set_A[0,1,:] = [-1, 1, 1] * unit.nanometers
set_A[1,1,:] = [-1, 0, 1] * unit.nanometers
set_A[2,1,:] = [0, 0, 1] * unit.nanometers

## coordenadas del tercer elemento de Set_A para los tres frames
set_A[0,2,:] = [-2, 0, 1] * unit.nanometers
set_A[1,2,:] = [-2, 0, 0] * unit.nanometers
set_A[2,2,:] = [-1, 1, 0] * unit.nanometers

# Set_B inicializado a cero:

set_B = np.zeros([3,2,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_B para los tres frames
set_B[0,0,:] = [4, -2, 0] * unit.nanometers
set_B[1,0,:] = [5, -2, -1] * unit.nanometers
set_B[2,0,:] = [5, -2, 0] * unit.nanometers

## coordenadas del segundo elemento de Set_B para los tres frames
set_B[0,1,:] = [3, 0, -1] * unit.nanometers
set_B[1,1,:] = [3, 1, 0] * unit.nanometers
set_B[2,1,:] = [4, 1, 1] * unit.nanometers

Podemos ahora invocando una sola vez el método `molmodmt.distance` calcular todas las distancias entre los elementos de ambos conjuntos para todos los frames:

In [ ]:
distances = m3t.distance(item_1=set_A, item_2=set_B)

De esta manera podemos por ejemplo acceder a la distancia entre el elemento 2-ésimo de `set_A` y el elemento 0-ésimo de `set_B` para el segundo frame o frame 1-ésimo.

In [ ]:
print(distances[1,2,0])

O imprimir esta distancia en función para todos los frames:

In [ ]:
print(distances[:,2,0])

Como vimos en el caso anterior, podemos ordenar que la salida del método `molmodmt.distance` sea en forma de diccionario de diccionarios.

In [ ]:
distances = m3t.distance(item_1=set_A, item_2=set_B, output_form='dict')

Podemos entonces acceder a las mismas distancias que anteriormente:

In [ ]:
# distancia entre el elemento 2-ésimo de set_A y 0-ésimo para el segundo frame
print(distances[2][0][1])

In [ ]:
# distancia entre el elemento 2-ésimo de set_A y 0-ésimo en cada frame
print(distances[2][0])

Esta manera de sacar los datos puede ser muy conveniente cuando calculamos distancias entre listas de elementos de ambos conjuntos. Supongamos que queremos calcular las distancias entre los elementos 1-ésimo y 2-ésimo de `set_A` y el elemento 1-ésimo de `set_B` para todos los frames. Primero veamos cómo hacerlo con la salida tipo tensor (salida por defecto):

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], item_2=set_B, selection_2=1)

De esta manera las distancias por ejemplo para ambos frames entre el elemento 2-ésimo de `set_A` con el elemento 1-ésimo de `set_B` se situan ahora en la posición 1-ésima (por ser el 1-ésimo elemento de la lista `selection_1`) frente a la 0-ésima (por ser el elemento 0-ésimo elemento, único en este caso, de la lista `selection_2`) en el segundo y tercer rango del tensor correspondientemente:

In [ ]:
print(distances[:,1,0])

La salida de `molmodmt.distances` tipo diccionario de diccionarios puede hacer más cómodo el trabajo con las distancias resultado, ya que el diccionario recupera los índices iniciales de cada elemento en ambos conjuntos `set_A` y `set_B`.

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], item_2=set_B, selection_2=1, output_form='dict')

La lista de distancias para los distintos frames entre el elemento 2-ésimo de `set_A` y el 1-ésimo de `set_B`, las mismas que anteriormente, puede ser encontrada según: 

In [ ]:
print(distances[2][1])

Por último, supongamos que únicamente queremos calcular la distancia entre estas dos últimas selecciones para el frame 0-ésimo y el 2-ésimo. Podemos incluir también como argumento de entrada en `molmodmt.distance` la selección de una lista de frames:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,2],
                         item_2=set_B, selection_2=1, frame_indices_2=[0,2])

Esta vez el resultado es un tensor donde ahora ya no tiene dimensión 3 en el primer rango, sino 2 (dos frames):

In [ ]:
distances.shape

Supongamos que queremos acceder a la distancia entre el elemento 2-ésimo de `set_A` con el elemento 1-ésimo de `set_B` en el frame 2-ésimo:

In [ ]:
print(distances[1,1,0])

Puedes comprobar que dado que `frame_indices_1` y `frame_indices_2` son iguales, puedes prescindir de `frame_indices_2`. El resultado será el mismo ya que el método sobreentiende que se trata de la misma selección de índices de frame cuando `frame_indices_2=None` y calcula las distancias entre las respectivas selecciones de elementos de ambos items en cada uno de los frames indicados únicamente en `frame_indices_1`:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,2],
                         item_2=set_B, selection_2=1)

In [ ]:
distances.shape

En esta situación en la que estamos incluyendo una lista de indices de frames, la salida de `molmodmt.distance` en forma de diccionario de diccionarios tambien respeta los índices originales en el caso de los frames:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,2],
                         item_2=set_B, selection_2=1, output_form='dict')

Ahora podemos conocer las distancias entre dos elementos en un frame determinado sin tener que acudir a los indices de dichos elementos y frame que tienen en las listas `selection_1`, `selection_2` y `frame_indices_1`. Basta con los índices absolutos originales que para el caso anterior son 2-ésimo de `set_A` y 1-ésimo de `set_B` en el frame 2-ésimo:

In [ ]:
print(distances[2][1][2])

## Distancias de desplazamiento o distancias entre distintas selecciones de índices de frame

La final de la sección anterior vimos cómo podemos hacer una selección de índices de frame cuando calculamos las distancias entre los elementos de dos modelos moleculares (objetos XYZ aquí). Podríamos ahora, por ejemplo, con lo aprendido hasta aquí calcular la distancia entre la selección `selection_1` del `item_1` en un cierto frame, por ejemplo el 0-ésimo, y la selección `selection_2` del `item_2` en otro cierto frame, por ejemplo el 2-ésimo:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=0,
                         item_2=set_B, selection_2=1, frame_indices_2=2)

In [ ]:
distances.shape

In [ ]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 2 es {}'.format(distances[0,1,0]))

O atendiendo a dos listas de índices de frames que serán enfrentadas secuencialmente: primer elemento de la primera lista frente a primer elemento de la segunda lista, segundo frente a segundo, y así consecutivamente:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,1],
                         item_2=set_B, selection_2=1, frame_indices_2=[1,2])

In [ ]:
distances.shape

In [ ]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 1 es {}'.format(distances[0,1,0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 2 es {}'.format(distances[1,1,0]))

O el mismo cálculo con su salida en forma de diccionario:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,1],
                         item_2=set_B, selection_2=1, frame_indices_2=[1,2], output_form='dict')

Donde dado que la relación entre índices de frame es uno a uno (el 0-ésimo de `set_A` frente al 1-ésimo de `set_B` y el 1-ésimo de `set_A` frente al 2-ésimo de `set_B`, en este caso), el diccionario resultante apela únicamente al índice original de `frame_indices_1`:

In [ ]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 1 es {}'.format(distances[2][1][0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 2 es {}'.format(distances[2][1][1]))

De esta manera podríamos incluso enfrentar ciertos elementos del set_A en todos sus frames contra ciertos elementos de set_B de un mismo frame, por ejemplo frente al 0-ésimo.

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], frame_indices_1=[0,1,2],
                         item_2=set_B, selection_2=1, frame_indices_2=[0,0,0])

In [ ]:
distances.shape

In [ ]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[0,2,0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[1,2,0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 2 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2,2,0]))

O igualmente:

In [ ]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], frame_indices_1=[0,1,2],
                         item_2=set_B, selection_2=1, frame_indices_2=[0,0,0], output_form='dict')

In [ ]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2][1][0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2][1][1]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 2 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2][1][2]))

Ya podemos entonces hacer el ejercicio de calcular distancias de desplazamiento de unos mismos elementos entre frames. Por ejemplo, veamos como podríamos responder la pregunta: ¿Cuanto se desplazan los elementos de `set_A` entre frames consecutivos?

In [ ]:
distances = m3t.distance(item_1=set_A, frame_indices_1=[0,1], frame_indices_2=[1,2])

Podemos ahora ver cuanto se desplazó por ejemplo el elemento 1-ésimo de `set_A` entre los frames 0-ésimo y 1-ésimo, o entre el 1-ésimo y el 2-esimo.

In [ ]:
print('La distancia de desplazamiento del elemento 1-ésimo de set_A entre los frames 0 y 1 es {}'.format(distances[0][1][1]))
print('La distancia de desplazamiento del elemento 1-ésimo de set_A entre los frames 1 y 2 es {}'.format(distances[1][1][1]))

El inconveniente que puede tener esta aproximación al cálculo de distancias de desplazamientos es que es absurdamente cara computacionalmente ya que también hemos tenido que calcular la distancia entre elementos distintos en frames distintos. O dicho de otra manera, para calcular la distancia de 3 elementos entre dos frames hemos tenido que calcular 9 distancias:

In [ ]:
distances.shape

No tienen en este último caso mucho sentido los elementos fuera de la diagonal de la matriz `distances[0,:,:]`. ¿Qué significa la distancia entre la posición del elemento 0-ésimo en el frame 0-ésimo y la posición del elemento 2-ésimo en el frame 1-ésimo, por ejemplo?

In [ ]:
distances[0,0,2] # En principio no tienen mucho sentido físicamente estos valores cruzados

Veamos en la siguiente sección como podríamos haber calculado el desplazamiento de los distintos elementos de `set_A` entre distintos frames sin necesidad de calcular los cruces entre distintos elementos del mismo objeto.

## Distancias entre pares

Cuando invocamos el método `molmodmt.distance` para calcular las distancias entre un conjunto de N elementos de `item_1` y M elementos de `item_2` el resultado es un objeto, tensor o diccionario, con NxM distancias por frame. Pero en ocasiones no necesitamos todas estas distancias sino que sólo queremos ciertas distancias entre pares dados por la lectura secuencial de los elementos de `selection_1` y `selection_2` uno a uno. Este cálculo se puede realizar introduciendo el argumento `pairs=True` en la llamada al método. Veamos esto mejor con un ejemplo recuperando los modelos `set_A` y `set_B`:

In [3]:
# Set_A inicializado a cero:
set_A = np.zeros([3,3,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_A para los tres frames 
set_A[0,0,:] = [0, 2, -1] * unit.nanometers
set_A[1,0,:] = [1, 2, -1] * unit.nanometers
set_A[2,0,:] = [0, 2, -1] * unit.nanometers

## coordenadas del segundo elemento de Set_A para los tres frames
set_A[0,1,:] = [-1, 1, 1] * unit.nanometers
set_A[1,1,:] = [-1, 0, 1] * unit.nanometers
set_A[2,1,:] = [0, 0, 1] * unit.nanometers

## coordenadas del tercer elemento de Set_A para los tres frames
set_A[0,2,:] = [-2, 0, 1] * unit.nanometers
set_A[1,2,:] = [-2, 0, 0] * unit.nanometers
set_A[2,2,:] = [-1, 1, 0] * unit.nanometers

# Set_B inicializado a cero:

set_B = np.zeros([3,2,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_B para los tres frames
set_B[0,0,:] = [4, -2, 0] * unit.nanometers
set_B[1,0,:] = [5, -2, -1] * unit.nanometers
set_B[2,0,:] = [5, -2, 0] * unit.nanometers

## coordenadas del segundo elemento de Set_B para los tres frames
set_B[0,1,:] = [3, 0, -1] * unit.nanometers
set_B[1,1,:] = [3, 1, 0] * unit.nanometers
set_B[2,1,:] = [4, 1, 1] * unit.nanometers

Calculemos en primer lugar algo sencillo para ilustrar este uso de `molmodmt.distance`. Calculemos la distancia entre los elementos 0-ésimo y 1-ésimo, y entre los elementos 0-ésimo y 2-ésimo de `set_A` para los frames 1-ésimo y 2-ésimo. 

In [11]:
distances = m3t.distance(item_1=set_A, selection_1=[0,0,1], selection_2=[0,2,2], frame_indices_1=[1,2], pairs=True)

La primera diferencia que podemos observar en el resultado frente al uso anterior de `molmodmt.distance` es el rango y dimensiones del tensor `distances`.

In [12]:
distances.shape

(2, 3)

El tensor es de rango 2, y no de rango 3 como anteriormente. Lo cual tiene mucho sentido porque efectivamente solo quisimos calcular 6 distancias correspondientes a tres pares de elementos para dos frames. Los pares son los construidos con el primer elemento de `selection_1` y el primer elemento de `selection_2`, el segundo de `selection_1` y el segundo de `selection_2`, y así sucesivamente. Es por eso que cuando `pairs=True` ambas selecciones deben tener el mísmo número de elementos. Una manera rápida de calcular los pares, si necesitas almacenarlos en una lista, es recurrir al iterador de python `zip`.

In [13]:
list_1 = [0,0,1]
list_2 = [0,2,2]
pairs = []

for ii,jj in zip(list_1,list_2):
    pairs.append([ii,jj])

print(pairs)

[[0, 0], [0, 2], [1, 2]]


Así, si necesitamos acudir al tensor resultante `distances` para ver la distancia del tercer par, en este caso [1,2], en el frame 1-ésimo:

In [14]:
print(distances[0,2])

1.4142135623730951 nm


De nuevo, el acceso a los datos resultantes puede ser un poco más cómodo si solicitamos que la salida tenga forma de diccionario de diccionarios. En este caso los indices originales de los elementos y los frames se respetan:

In [16]:
distances = m3t.distance(item_1=set_A, selection_1=[0,0,1], selection_2=[0,2,2], frame_indices_1=[1,2],
                         pairs=True, output_form='dict')

Veamos ahora la misma distancia entre el elemento 1-ésimo y el 2-ésimo para el frame 1-ésimo:

In [19]:
print(distances[1][2][1])

1.4142135623730951 nm


Esta opción `pairs=True` nos permite entonces calcular las distancias de desplazamiento de los elementos de `set_A` entre frames consecutivos de una manera más económica computacionalmente (a diferencia de lo observado en el final de la sección anterior):

In [22]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], selection_2=[0,1,2],
                         frame_indices_1=[0,1], frame_indices_2=[1,2], pairs=True)

Sólo hemos tenido que calcular los 6 desplazamientos de interes, y no los 18 que fueron necesarios cuando `pairs=False` (opción por defecto). Veamos por ejemplo el desplazamiento del elemento 2-ésimo entre el frame 1-ésimo y el 2-ésimo:

In [23]:
print(distances[1,2])

1.4142135623730951 nm


De nuevo con la opción `output_form` el acceso a los desplazamientos se hace con los índices originales:

In [26]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], selection_2=[0,1,2],
                         frame_indices_1=[0,1], frame_indices_2=[1,2],
                         pairs=True, output_form='dict')

Así entonces, el mismo desplazamiento del 2-ésimo elemento entre el frame 1-ésimo y el 2-ésimo:

In [28]:
print(distances[2][2][1])

1.4142135623730951 nm


## Distancias máximas y mínimas

En múchas ocasiones lo que necesitamos calcular los distancias mínimas o distancias máximas. Para facilitar la obtención de estos observables encontramos en MolModMT dos métodos específicos cuya manera de ser usados es similar: `molmodmt.minimum_distance` y `molmodmt.maximum_distance`. Trabajemos en este sección con el cálculo de distancias mínimas para ilustrar el funcionamiento de estos métodos.

Recuperemos los mismos modelos usados anteriormente, `set_A` y `set_B`, con tres frames de coordenadas espaciales cada uno:

In [22]:
# Set_A inicializado a cero:
set_A = np.zeros([3,3,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_A para los tres frames 
set_A[0,0,:] = [0, 2, -1] * unit.nanometers
set_A[1,0,:] = [1, 2, -1] * unit.nanometers
set_A[2,0,:] = [0, 2, -1] * unit.nanometers

## coordenadas del segundo elemento de Set_A para los tres frames
set_A[0,1,:] = [-1, 1, 1] * unit.nanometers
set_A[1,1,:] = [-1, 0, 1] * unit.nanometers
set_A[2,1,:] = [0, 0, 1] * unit.nanometers

## coordenadas del tercer elemento de Set_A para los tres frames
set_A[0,2,:] = [-2, 0, 1] * unit.nanometers
set_A[1,2,:] = [-2, 0, 0] * unit.nanometers
set_A[2,2,:] = [-1, 1, 0] * unit.nanometers

# Set_B inicializado a cero:

set_B = np.zeros([3,2,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_B para los tres frames
set_B[0,0,:] = [4, -2, 0] * unit.nanometers
set_B[1,0,:] = [5, -2, -1] * unit.nanometers
set_B[2,0,:] = [5, -2, 0] * unit.nanometers

## coordenadas del segundo elemento de Set_B para los tres frames
set_B[0,1,:] = [3, 0, -1] * unit.nanometers
set_B[1,1,:] = [3, 1, 0] * unit.nanometers
set_B[2,1,:] = [4, 1, 1] * unit.nanometers

Calculemos en primer lugar la distancia mínima entre los elementos de `set_A` y los de `set_B` para cada uno de los tres frames:

In [29]:
min_distances = m3t.minimum_distance(item_1=set_A, item_2=set_B)

ValueError: Value of argument group_behavior_1 not recognized.

## Distancias entre grupos 

## Listas de vecinos

## Mapas de contactos

In [ ]:
molsys = m3t.load('/home/diego/1brs.pdb', selection='chainid 0 or chainid 3')

In [ ]:
m3t.view(molsys)

In [ ]:
dists_CAs = m3t.distance(molsys, selection_1="chainid 0 and name CA", selection_2="chainid 3 and name CA")

In [ ]:
chain0_CAs = m3t.select(molsys, "chainid 0 and name CA")
chain3_CAs = m3t.select(molsys, "chainid 3 and name CA")

ii=10
jj=20
frame_index=0

print('La distancia entre el átomo {} y el átomo {} es {}'.format(chain0_CAs[ii], chain3_CAs[jj], dists_CAs[frame_index,ii,jj]))

In [ ]:
dists_CAs = m3t.distance(molsys, selection_1="chainid 0 and name CA", selection_2="chainid 3 and name CA", output_form='dict')

In [ ]:
ii=74
jj=1022
frame_index=0

print('La distancia entre el átomo {} y el átomo {} es {}'.format(ii, jj, dists_CAs[ii][jj][frame_index]))

In [ ]:
m3t.get(molsys, target='atom', indices=[74,1022], index=True, id=True, name=True, element=True)